# EJERCICIO 2
APRENDIZAJE POR REFUERZO PARA QUE UN AGENTE PUEDA JUGAR 4 EN RAYA

In [2]:
import numpy as np
from tqdm import tqdm
import pandas as pd
import pickle

Modelando el entorno de juego: inicializa la matriz 6×7 que representa el tablero.

In [3]:
class Board():
    def __init__(self, rows=6, cols=7):
        self.rows, self.cols = rows, cols
        self.state = np.zeros((rows, cols), dtype=int)
    #Calcula y devuelve la lista de columnas donde aún se puede soltar una ficha.
    def valid_moves(self):
        # columnas que no están llenas (si la fila 0 está vacía)
        return [c for c in range(self.cols) if self.state[0, c] == 0]
    #Simula la gravedad: coloca la ficha en la fila vacía más baja de la columna elegida.
    def update(self, symbol, col):
        # coloca la ficha en la fila más baja disponible de la columna
        for r in range(self.rows-1, -1, -1):
            if self.state[r, col] == 0:
                self.state[r, col] = symbol
                return
        raise ValueError("Columna llena, movimiento ilegal!")

    #Revisa líneas horizontales, verticales y diagonales para ver si hay 4 en raya o empate
    def is_game_over(self):
        S = self.state
        # checks para cada posición no cero
        for r in range(self.rows):
            for c in range(self.cols):
                player = S[r,c]
                if player == 0:
                    continue
                # direcciones: (dr, dc)
                for dr, dc in [(0,1), (1,0), (1,1), (1,-1)]:
                    end_r = r + 3*dr
                    end_c = c + 3*dc
                    if 0 <= end_r < self.rows and 0 <= end_c < self.cols:
                        # comprueba las 4 en línea
                        if all(S[r + i*dr, c + i*dc] == player for i in range(4)):
                            return player  # 1 o -1
        # empate
        if len(self.valid_moves()) == 0:
            return 0
        # no terminal
        return None
    #Limpia el tablero dejándolo todo en cero para una nueva partida
    def reset(self):
        self.state.fill(0)

In [4]:
class Game():
    #Crea la instancia de Board, asigna símbolos (+1/–1) a cada agente y prepara el flujo de turnos.
    def __init__(self, player1, player2, rows=6, cols=7):
        player1.symbol, player2.symbol = 1, -1
        self.players = [player1, player2]
        self.board = Board(rows, cols)

    #Ejecuta N rondas: alterna turnos, actualiza posiciones y acumula victorias
    def selfplay(self, rounds=100):
        wins = [0, 0]
        for _ in tqdm(range(rounds)):
            self.board.reset()
            for p in self.players:
                p.reset()
            game_over = False
            while not game_over:
                for p in self.players:
                    action = p.move(self.board)
                    self.board.update(p.symbol, action)
                    p.update(self.board)   # registra estado
                    if self.board.is_game_over() is not None:
                        game_over = True
                        break
            self.reward()
            result = self.board.is_game_over()
            if result != 0:
                idx = 0 if result == self.players[0].symbol else 1
                wins[idx] += 1
        return wins

    #Otorga 1 punto al ganador, 0.5 a cada uno en empate, y propaga esas recompensas a los agentes
    def reward(self):
        winner = self.board.is_game_over()
        if winner == 0:
            for p in self.players:
                p.reward(0.5)
        else:
            for p in self.players:
                p.reward(1 if winner == p.symbol else 0)

In [5]:
class Agent():
    #Crea la tabla de valores vacía, define la tasa de aprendizaje y la probabilidad de exploración
    def __init__(self, alpha=0.5, prob_exp=0.5):
        self.value_function = {}       # estado (str) -> valor
        self.alpha = alpha
        self.prob_exp = prob_exp
        self.positions = []

    def reset(self):
        self.positions = []

    #Con probabilidad ε elige movimiento aleatorio; otherwise elige la columna con mayor valor estimado
    def move(self, board, explore=True):
        valid_cols = board.valid_moves()
        # exploración ε-greedy
        if explore and np.random.rand() < self.prob_exp:
            return np.random.choice(valid_cols)
        # explotación: pruebo cada columna y elijo la que maximice value
        best_col, max_val = None, -np.inf
        for c in valid_cols:
            tmp = board.state.copy()
            # simulo la jugada
            for r in range(board.rows-1, -1, -1):
                if tmp[r, c] == 0:
                    tmp[r, c] = self.symbol
                    break
            st = str(tmp.reshape(board.rows * board.cols))
            val = self.value_function.get(st, 0)
            if val > max_val:
                max_val, best_col = val, c
        return best_col

    #Guarda en la memoria cada estado de tablero generado tras el movimiento
    def update(self, board):
        st = str(board.state.reshape(board.rows * board.cols))
        self.positions.append(st)

    #Recorre los estados guardados en orden inverso y aplica un TD-like update para aprender
    def reward(self, reward):
        for st in reversed(self.positions):
            if st not in self.value_function:
                self.value_function[st] = 0
            # TD(0)-like update
            self.value_function[st] += self.alpha * (reward - self.value_function[st])
            reward = self.value_function[st]

EJECUCION DE ENTRENAMIENTO
Crea 2 agentes y egecuta para 100000 rondas

In [6]:
agent1 = Agent(prob_exp=0.5)
agent2 = Agent(prob_exp=0.5)
game = Game(agent1, agent2, rows=6, cols=7)

wins = game.selfplay(100000)
print("Victorias:", wins)

100%|██████████| 100000/100000 [21:41<00:00, 76.84it/s]

Victorias: [58251, 41304]


In [7]:
#Función de valor: lista de tuplas (estado, valor) ordenada de mayor a menor
funcion_de_valor = sorted(agent1.value_function.items(),key=lambda kv: kv[1], reverse=True)
tabla = pd.DataFrame({ 'estado': [x[0] for x in funcion_de_valor],'valor': [x[1] for x in funcion_de_valor]})

tabla

,estado,valor
0,[ 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,1.0
1,[ 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,1.0
2,[ 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,1.0
3,[ 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,1.0
4,[ 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,1.0
...,...,...
602788,[-1 0 0 0 0 0 0 1 1 0 0 0 0 0 -1 ...,0.0
602789,[-1 0 0 0 0 0 0 1 0 0 0 0 0 0 -1 ...,0.0
602790,[-1 0 0 0 0 0 0 1 0 0 0 0 0 0 -1 ...,0.0
602791,[ 0 0 0 0 0 0 0 1 0 0 0 0 0 0 -1 ...,0.0


In [8]:
#Serializamos el diccionario de valores a un archivo pickle
with open('agente_connect4.pickle', 'wb') as f:
    pickle.dump(agent1.value_function, f)